<a href="https://colab.research.google.com/github/nngiabao/MLPractice/blob/main/DigitRecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MNISTDigit Recognition
#CNN Deep Learning
#Link: https://www.kaggle.com/code/ibrahimqasimi/mnist-digit-recognition-cnn-deep-learning

In [ ]:
#data handling and numerical computatioms
import numpy as np
import pandas as pd
#data visualization
import matplotlib.pyplot as plt
import seaborn as sns
#load MNIST dataset from Keras
from tensorflow.keras.datasets import mnist
#

In [ ]:
#spliting into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#Display dataset shapes
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
#
num_epochs = 100
running_loss = 0.0
#
for epoch in range(num_epochs):
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()